In [ ]:
import cv2
import numpy as np
import os

In [ ]:
cap = cv2.VideoCapture(0)
"""
if not os.path.exists("imgdata"):
    os.makedirs("imgdata")
    if not os.path.exists("imgdata/train"):
    os.makedirs("imgdata/train")
    if not os.path.exists("imgdata/test"):
    os.makedirs("imgdata/test")
    if not os.path.exists("imgdata/train/backward"):
    os.makedirs("imgdata/train/backward")
    if not os.path.exists("imgdata/train/forward"):
    os.makedirs("imgdata/train/forward")
    if not os.path.exists("imgdata/train/left"):
    os.makedirs("imgdata/train/left")
    if not os.path.exists("imgdata/train/right"):
    os.makedirs("imgdata/train/right")
    if not os.path.exists("imgdata/train/backward"):
    os.makedirs("imgdata/test/backward")
    if not os.path.exists("imgdata/test/forward"):
    os.makedirs("imgdata/test/forward")
    if not os.path.exists("imgdata/test/left"):
    os.makedirs("imgdata/test/left")
    if not os.path.exists("imgdata/test/right"):
    os.makedirs("imgdata/test/right")
"""

if not os.path.exists("imgdata"):
    os.makedirs("imgdata")  
    os.makedirs("imgdata/train")
    os.makedirs("imgdata/test")
    os.makedirs("imgdata/train/backward")
    os.makedirs("imgdata/train/forward")
    os.makedirs("imgdata/train/left")
    os.makedirs("imgdata/train/right")
    os.makedirs("imgdata/test/backward")
    os.makedirs("imgdata/test/forward")
    os.makedirs("imgdata/test/left")
    os.makedirs("imgdata/test/right")
    
    
    
mode = 'train'
   
       

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    
    directory = 'imgdata/'+mode+'/' 
    # Getting count of existing images
    count = {'backward': len(os.listdir(directory+"/backward")),
             'forward': len(os.listdir(directory+"/forward")),
             'left': len(os.listdir(directory+"/left")),
             'right': len(os.listdir(directory+"/right"))}
 
    
    # Printing the count in each set to the screen
    cv2.putText(frame, "press q for change mode:"+mode, (10, 100), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "BACKWARD :"+str(count['backward']), (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "FORWARD :"+str(count['forward']), (10, 140), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "LEFT :"+str(count['left']), (10, 160), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
    cv2.putText(frame, "RIGHT :"+str(count['right']), (10, 180), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)
  

    

    x1 = int(0.5*frame.shape[1])
    y1 = 60
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])-10
   
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
   
    roi = frame[y1:y2, x1:x2]
    roi = cv2.resize(roi, (64, 64)) 
 
    cv2.imshow("Frame", frame)
    
    #_, mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(mask, kernel, iterations=1)
    #img = cv2.erode(mask, kernel, iterations=1)
    # do the processing after capturing the image!
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, roi = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("ROI", roi)
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
    if interrupt & 0xFF == ord('s'):
        cv2.imwrite(directory+'backward/'+str(count['backward'])+'.jpg', roi)
    if interrupt & 0xFF == ord('w'):
        cv2.imwrite(directory+'forward/'+str(count['forward'])+'.jpg', roi)
    if interrupt & 0xFF == ord('a'):
        cv2.imwrite(directory+'left/'+str(count['left'])+'.jpg', roi)
    if interrupt & 0xFF == ord('d'):
        cv2.imwrite(directory+'right/'+str(count['right'])+'.jpg', roi)
    if interrupt & 0xFF == ord('q'):
        if mode == "train":
            mode="test"
        else:
            mode="train"
            
cap.release()
cv2.destroyAllWindows()


In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=128, activation='sigmoid'))
model.add(Dense(units=4, activation='softmax')) 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255, #normalization here
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255) #normalization here

training_set = train_datagen.flow_from_directory('imgdata/train',
                                                 target_size=(64, 64),
                                                 batch_size=5,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('imgdata/test',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
test_set.labels

In [ ]:

#little testing for myself not necessary skip this
train_path = "imgdata/train/"
test_path = "imgdata/test/"

classes=0
train_num =0
test_num = 0

for i in os.listdir(train_path):
    classes+=1
    
for i in range(0,classes):
    for img in os.listdir(train_path+str(i)): 
        train_num+=1

classes=0        
for i in os.listdir(test_path):
    classes+=1
    
for i in range(0,classes):
    for img in os.listdir(test_path+str(i)): 
        test_num+=1
print(train_num,test_num)



In [ ]:
model.fit_generator(
        training_set,
        steps_per_epoch=400,
        epochs=5,
        validation_data=test_set,
        validation_steps=20)


In [ ]:
# Saving the model
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model-bw.h5')



# Loading the model
json_file = open("model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")

In [ ]:
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys, os

cap = cv2.VideoCapture(0)

categories = {0: 'BACKWARD', 1: 'FORWARD', 2: 'LEFT', 3: 'RIGHT'}

while True:
    _, frame = cap.read()
    # Simulating mirror image
    frame = cv2.flip(frame, 1)
    

    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])
   

    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)

    roi = frame[y1:y2, x1:x2]
    

    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("test", test_image)
    
    
    result = model.predict(test_image.reshape(1, 64, 64, 1))
    
    prediction = {'BACKWARD': result[0][0], 
                  'FORWARD': result[0][1], 
                  'LEFT': result[0][2],
                  'RIGHT': result[0][3]}

    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    
    # Displaying the predictions
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
    cv2.imshow("Frame", frame)
    
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
        
cap.release()
cv2.destroyAllWindows()
